In [1]:
import pandas as pd
from utils import Timer
import featuretools as ft
import pandas as pd
import numpy as np
from woodwork.logical_types import Categorical, Datetime, Double
from woodwork.column_schema import ColumnSchema
from woodwork.logical_types import Boolean, BooleanNullable
import lightgbm as lgbm
from pandas.api import types as pdt

In [2]:
file = "/mnt/DP_disk1/ht/datasets/autofe/tabutils/Covid_19_ICU_prediction.csv"
with Timer("read train data"):
    # Reading Data
    df =  pd.read_csv(file)

values = {}
for col in df.columns:
    if  pdt.is_numeric_dtype(df[col]):
        values[col] = 0
df.fillna(values, inplace=True)
df

read train data took 0.05821171496063471 sec


,PATIENT_VISIT_IDENTIFIER,AGE_ABOVE65,AGE_PERCENTIL,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,...,TEMPERATURE_DIFF,OXYGEN_SATURATION_DIFF,BLOODPRESSURE_DIASTOLIC_DIFF_REL,BLOODPRESSURE_SISTOLIC_DIFF_REL,HEART_RATE_DIFF_REL,RESPIRATORY_RATE_DIFF_REL,TEMPERATURE_DIFF_REL,OXYGEN_SATURATION_DIFF_REL,WINDOW,ICU
0,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,0
1,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,2-4,0
2,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4-6,0
3,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,...,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,6-12,0
4,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,...,-0.238095,-0.818182,-0.389967,0.407558,-0.230462,0.096774,-0.242282,-0.814433,ABOVE_12,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1920,384,0,50th,1,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,0
1921,384,0,50th,1,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,2-4,0
1922,384,0,50th,1,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,4-6,0
1923,384,0,50th,1,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,6-12,0


In [3]:
with Timer("Load data to entityset"):
    es = ft.EntitySet("cdp")
    es.add_dataframe(dataframe_name="cdp",
                     dataframe=df,
                     index="id", 
                     )

with Timer("DFS feature generation"):
    ret_df, features = ft.dfs(
        entityset=es, 
        target_dataframe_name="cdp", 
        verbose=True,
        )

ret_df.info()

/root/anaconda3/envs/autofe/lib/python3.8/site-packages/featuretools/entityset/entityset.py:1910: UserWarning: index id not found in dataframe, creating new integer column
  warnings.warn(
/root/anaconda3/envs/autofe/lib/python3.8/site-packages/featuretools/synthesis/deep_feature_synthesis.py:169: UserWarning: Only one dataframe in entityset, changing max_depth to 1 since deeper features cannot be created
  warnings.warn(


Load data to entityset took 0.39734547305852175 sec
Built 231 features
Elapsed: 00:00 | Progress: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████
DFS feature generation took 0.2825850062072277 sec
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1925 entries, 0 to 1924
Columns: 231 entries, PATIENT_VISIT_IDENTIFIER to ICU
dtypes: category(2), float64(225), int64(4)
memory usage: 3.4 MB


In [4]:
params = {
    'boosting_type':'gbdt',
    'objective': 'binary',
    'num_leaves': 31,
    'learning_rate': 0.01,
    'seed':0,
    'verbose': 1,
    'metrics': 'auc'
}
target_label = 'ICU'
valid_sample = ret_df.sample(frac = 0.1)
train_sample = ret_df.drop(valid_sample.index)
lgbm_train = lgbm.Dataset(train_sample.drop(columns=[target_label]), train_sample[target_label])
lgbm_val = lgbm.Dataset(valid_sample.drop(columns=[target_label]), valid_sample[target_label])

with Timer("LGBM training"):
    model = lgbm.train(params=params, train_set=lgbm_train, valid_sets=lgbm_val, verbose_eval=100)

/root/anaconda3/envs/autofe/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
/root/anaconda3/envs/autofe/lib/python3.8/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/root/anaconda3/envs/autofe/lib/python3.8/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 463, number of negative: 1270
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.016121 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16420
[LightGBM] [Info] Number of data points in the train set: 1733, number of used features: 230
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.267167 -> initscore=-1.009045
[LightGBM] [Info] Start training from score -1.009045
[100]	valid_0's auc: 0.89794
LGBM training took 1.1687060687690973 sec
